In [1]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/PBL7/Tram

Mounted at /content/drive
/content/drive/MyDrive/PBL7/Tram


In [3]:
!pip install cloudscraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.9 MB/s eta 0:00:00


## 1. Crawl Journal (to get Ratings)

In [ ]:
book_detail = pd.read_csv('/content/drive/MyDrive/PBL7/Tram/BookCrossingData/book_detail.csv', index_col=False)
print(book_detail.shape)
book_detail.head()

(126280, 8)
(126280, 8)


,isbn,url,title,image_url,author,category,avg_rating,book_list_id
0,0889610487,https://www.bookcrossing.com/title/6330328,One Proud Summer,https://books.google.com/books/content?id=NxTh...,Marsha Hewitt,Children's Books,8.0,0
1,0394280741,https://www.bookcrossing.com/title/1712457,The Hollow Tree,https://books.google.com/books/content?id=Vaov...,Janet Lunn,Children's Books,8.0,0
2,0773770208,https://www.bookcrossing.com/title/1221204,One Man's Island,https://images-na.ssl-images-amazon.com/images...,David Conover,Literature & Fiction,NaN,0
3,0888330162,https://www.bookcrossing.com/title/6330373,Outdoors West,https://images-na.ssl-images-amazon.com/images...,Ian Bickle,Outdoors & Nature,8.0,0
4,155054442X,https://www.bookcrossing.com/title/6134748,The Secret Lives of Sgt. John Wilson,https://books.google.com/books/content?id=uQzD...,Lois Simmie,History,7.0,0
...,...,...,...,...,...,...,...,...
126275,1869712943,https://www.bookcrossing.com/title/6551706,Jonah,/images/book-title-not-found_270x400.jpg,Jonah Lomu,Biographies & Memoirs,NaN,1169
126276,1921793082,https://www.bookcrossing.com/title/6110762,The Iron Queen,/images/book-title-not-found_270x400.jpg,Julie Kagawa,Teens,NaN,1169
126277,0755309278,https://www.bookcrossing.com/title/203971,Belfast Confidential,/images/book-title-not-found_270x400.jpg,Bateman,Mystery & Thrillers,7.0,1169
126278,0330418688,https://www.bookcrossing.com/title/418862,Dead Ringer,/images/book-title-not-found_270x400.jpg,Lisa Scottoline,Mystery & Thrillers,NaN,1169


In [ ]:
# Remove rows with missing avg_rating
book_detail = book_detail[book_detail['avg_rating'].notna()]
book_detail = book_detail[["isbn", "url"]]
print(book_detail.shape)
book_detail

(32340, 2)


,isbn,url
0,0889610487,https://www.bookcrossing.com/title/6330328
1,0394280741,https://www.bookcrossing.com/title/1712457
3,0888330162,https://www.bookcrossing.com/title/6330373
4,155054442X,https://www.bookcrossing.com/title/6134748
5,0771066937,https://www.bookcrossing.com/title/2501125
...,...,...
126260,0340231203,https://www.bookcrossing.com/title/1076098
126265,0006134181,https://www.bookcrossing.com/title/2688911
126266,0099507659,https://www.bookcrossing.com/title/6317198
126273,1407159283,https://www.bookcrossing.com/title/5629838


In [ ]:
current_index = 0
book_detail = book_detail[current_index:]
print(book_detail.shape)
book_detail

(32340, 2)


,isbn,url
0,0889610487,https://www.bookcrossing.com/title/6330328
1,0394280741,https://www.bookcrossing.com/title/1712457
3,0888330162,https://www.bookcrossing.com/title/6330373
4,155054442X,https://www.bookcrossing.com/title/6134748
5,0771066937,https://www.bookcrossing.com/title/2501125
...,...,...
126260,0340231203,https://www.bookcrossing.com/title/1076098
126265,0006134181,https://www.bookcrossing.com/title/2688911
126266,0099507659,https://www.bookcrossing.com/title/6317198
126273,1407159283,https://www.bookcrossing.com/title/5629838


In [ ]:
# create journal_link.csv
import pandas as pd
pd.DataFrame(columns=['isbn', 'journal_url']).to_csv('journal_link.csv', mode='w', header=True, index=False)

In [ ]:
import time
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd

journal_urls = []
max_attempts = 10
scraper = cloudscraper.create_scraper()

def save_links_to_csv(journal_link):
    journal_link_df = pd.DataFrame(journal_link, columns=['isbn', 'journal_url'])
    journal_link_df.to_csv('journal_link.csv', mode='a', header=False, index=False)

for isbn, url in book_detail.itertuples(index=False):
    for attempt in range(max_attempts):
        try:
            # Lấy nội dung trang web
            response = scraper.get(url)

            # Kiểm tra nếu yêu cầu thành công
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                div_tag = soup.find('div', id="dCopies")
                if div_tag:
                    journal_links = div_tag.find_all('a', href=True)
                    journal_urls.append([isbn, [a['href'] for a in journal_links if '/journal/' in a['href']]])
                    if len(journal_urls) == 100:
                        save_links_to_csv(journal_urls)
                        current_index += 100
                        print(f"Saved journal links in 100 books. Current Index: {current_index}.")
                        journal_urls = []
                break
            else:
                print(f"Không thể truy cập trang web, mã trạng thái: {response.status_code}")
                if attempt == max_attempts - 1:
                    print(f"URL: {url} sẽ được bỏ qua.")
            # Chờ 3 giây trước khi thử lại
            time.sleep(3)
        except Exception as e:
            print(f"Exception occurred: {e}")
        finally:
            scraper.close()

# Save any remaining journal URLs
if journal_urls:
    save_links_to_csv(journal_urls)
    print(f"Saved remaining journal links. Total saved: {current_index + len(journal_urls)}.")

Saved journal links in 100 books. Current Index: 100.
Saved journal links in 100 books. Current Index: 200.
Saved journal links in 100 books. Current Index: 300.
Saved journal links in 100 books. Current Index: 400.
Saved journal links in 100 books. Current Index: 500.
Saved journal links in 100 books. Current Index: 600.
Saved journal links in 100 books. Current Index: 700.
Saved journal links in 100 books. Current Index: 800.
Saved journal links in 100 books. Current Index: 900.
Saved journal links in 100 books. Current Index: 1000.
Saved journal links in 100 books. Current Index: 1100.
Saved journal links in 100 books. Current Index: 1200.
Saved journal links in 100 books. Current Index: 1300.
Saved journal links in 100 books. Current Index: 1400.
Saved journal links in 100 books. Current Index: 1500.
Saved journal links in 100 books. Current Index: 1600.
Saved journal links in 100 books. Current Index: 1700.
Saved journal links in 100 books. Current Index: 1800.
Saved journal links

## 2. Crawl Ratings

In [4]:
journal_urls = pd.read_csv('/content/drive/MyDrive/PBL7/Tram/journal_link.csv', index_col=False)
print(journal_urls.shape)
journal_urls

(32340, 2)


,isbn,journal_url
0,0889610487,['/journal/16531299']
1,0394280741,"['/journal/459602', '/journal/16531285', '/jou..."
2,0888330162,['/journal/16531351']
3,155054442X,"['/journal/4050693', '/journal/16182680', '/jo..."
4,0771066937,"['/journal/429761', '/journal/3310381', '/jour..."
...,...,...
32335,0340231203,"['/journal/6789933', '/journal/846254']"
32336,0006134181,"['/journal/2734694', '/journal/1966431', '/jou..."
32337,0099507659,['/journal/16249455']
32338,1407159283,['/journal/15338880']


In [5]:
current_book_with_journal_index = 25200
journal_urls = journal_urls[current_book_with_journal_index:]
print(journal_urls.shape)
journal_urls

(7140, 2)


,isbn,journal_url
25200,0583110665,['/journal/3526054']
25201,0340766751,['/journal/3461009']
25202,0333751736,['/journal/3261425']
25203,0853406316,['/journal/12403867']
25204,1841953512,"['/journal/4734922', '/journal/2875735', '/jou..."
...,...,...
32335,0340231203,"['/journal/6789933', '/journal/846254']"
32336,0006134181,"['/journal/2734694', '/journal/1966431', '/jou..."
32337,0099507659,['/journal/16249455']
32338,1407159283,['/journal/15338880']


In [ ]:
# create user_ratings.csv
# import pandas as pd
# pd.DataFrame(columns=['isbn', 'user_url', 'rating', 'rating_time']).to_csv('user_ratings.csv', mode='w', header=True, index=False)

In [6]:
import time
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re

user_ratings = []
base_url = 'https://bookcrossing.com'
max_attempts = 10
scraper = cloudscraper.create_scraper()

def save_links_to_csv(user_ratings):
    user_ratings_df = pd.DataFrame(user_ratings, columns=['isbn', 'user_url', 'rating', 'rating_time'])
    user_ratings_df.to_csv('user_ratings.csv', mode='a', header=False, index=False)

for isbn, journal_url in journal_urls.itertuples(index=False):
    journal_url = re.sub(r"[\[\]',]", '', journal_url)
    url_list = journal_url.split(' ')
    current_book_with_journal_index += 1
    for current_url in url_list:
        for attempt in range(max_attempts):
            try:
                # Lấy nội dung trang web
                url = base_url + current_url
                response = scraper.get(url)
                # Kiểm tra nếu yêu cầu thành công
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    review_divs = soup.find_all('div', style="padding-left: 65px")

                    if review_divs:
                        for div in review_divs:
                            rating_tag = div.find('span', class_='bookrating')
                            if rating_tag:
                                user_tag = div.find('a', title=True, href=lambda href: href and '/mybookshelf/' in href)
                                user_url = user_tag['href']
                                rating_title = rating_tag['title']
                                rating_value = rating_title.split(' ')[0]
                                review_info = div.find('div', class_='my-1 small').text
                                rating_time_str = review_info.split(' on ')[-1].strip()
                                rating_time = datetime.strptime(rating_time_str, '%A, %B %d, %Y').isoformat()

                                user_ratings.append((isbn, user_url, rating_value, rating_time))
                    break
                else:
                    print(f"Không thể truy cập trang web, mã trạng thái: {response.status_code}")
                    if attempt == max_attempts - 1:
                        print(f"URL: {url} se duoc bo qua.")

                # Chờ 3 giây trước khi thử lại
                time.sleep(3)
            except Exception as e:
                print(f"Exception occurred: {e}")
            finally:
                scraper.close()
    if current_book_with_journal_index % 100 == 0:
        save_links_to_csv(user_ratings)
        print(f"Saved {len(user_ratings)} user ratings. Current Index: {current_book_with_journal_index}.")
        user_ratings = []

# Save any remaining user ratings
if user_ratings:
    save_links_to_csv(user_ratings)
    print(f"Saved {len(user_ratings)} remaining user ratings. Total saved: {current_book_with_journal_index}.")

Saved 692 user ratings. Current Index: 25300.
Saved 647 user ratings. Current Index: 25400.
Saved 563 user ratings. Current Index: 25500.
Saved 756 user ratings. Current Index: 25600.
Saved 643 user ratings. Current Index: 25700.
Saved 359 user ratings. Current Index: 25800.
Saved 646 user ratings. Current Index: 25900.
Saved 254 user ratings. Current Index: 26000.
Saved 386 user ratings. Current Index: 26100.
Saved 531 user ratings. Current Index: 26200.
Saved 762 user ratings. Current Index: 26300.
Saved 785 user ratings. Current Index: 26400.
Saved 343 user ratings. Current Index: 26500.
Saved 372 user ratings. Current Index: 26600.
Saved 250 user ratings. Current Index: 26700.
Saved 147 user ratings. Current Index: 26800.
Saved 345 user ratings. Current Index: 26900.
Saved 731 user ratings. Current Index: 27000.
Saved 676 user ratings. Current Index: 27100.
Saved 412 user ratings. Current Index: 27200.
Saved 170 user ratings. Current Index: 27300.
Saved 286 user ratings. Current In

## 3. Crawl User Profiles

In [ ]:
# read user ratings
user_ratings_df = pd.read_csv('user_ratings.csv')
print(user_ratings_df.shape)
user_ratings_df

(119917, 4)


,isbn,user_url,rating,rating_time
0,0889610487,/mybookshelf/_Figment_/,8,2022-04-17T00:00:00
1,0394280741,/mybookshelf/Ella-the-Elf/,7,2003-01-11T00:00:00
2,0394280741,/mybookshelf/AnonymousFinder/,7,2011-04-08T00:00:00
3,0394280741,/mybookshelf/_Figment_/,8,2022-04-17T00:00:00
4,0394280741,/mybookshelf/redhead344/,8,2002-11-17T00:00:00
...,...,...,...,...
119912,0552998052,/mybookshelf/birminghamboris/,9,2021-10-31T00:00:00
119913,0552998052,/mybookshelf/ifyouknew/,7,2012-12-11T00:00:00
119914,0552998052,/mybookshelf/Hawkette/,7,2019-10-23T00:00:00
119915,0552998052,/mybookshelf/catsalive/,8,2019-11-05T00:00:00


In [ ]:
# prompt: count number of different isbn in user ratings

unique_isbns = set(user_ratings_df['isbn'])
print(f"Number of unique ISBNs in user ratings: {len(unique_isbns)}")


Number of unique ISBNs in user ratings: 19791
